In [ ]:
using Plots
include("integrators/etd.jl");

In [ ]:
const N = 5_000
const L = 1_000
const dt = 1e-2
const α = .6
const par = setup(N, L, dt, α);
const g = get_g(par);

In [ ]:
x, F = par.x, par.F

v = .6
A = 1e-2
φ1 = v * ones(N) + A* cos.((2π/L).*x)
φ2 = A * sin.((2π/L).*x); 
Fφ1, Fφ2 = F*φ1, F*φ2;

In [ ]:
M = Int(2e6)
frames = 5_000
n = M ÷ frames

φt = Array{Float64}(undef, (frames, 2, N))
φt[1, 1, :] .= φ1
φt[1, 2, :] .= φ2;

In [ ]:
for i in 2:frames
    for j in 1:n
        etd!(par, g, (Fφ1, Fφ2), (φ1, φ2))
        t = i*n + j
        if t%(M÷10) == 0
            print(100*t/M,"% \n")
            @assert !any(isnan.(Fφ1))
        end
    end
    φt[i, 1, :] .= φ1
    φt[i, 2, :] .= φ2
end

In [ ]:
 anim = @animate for i in 1:frames
    plot()
    plot!(x, φt[i, 1, :])
    plot!(x, φt[i, 2, :])
    plot!(;yrange=(-1, 1))
    plot!(size=(1500, 500))
end every 1;
gif(anim, "nrch.mp4", fps = 15)